In [ ]:
# User inputs
ndays = "4d"  # Number of days/hours to query
client_name = "usdf_efd"  # EFD client name
path = "auxtel/correct_pointing.py"  # Script path
salIndex = 2  # Script Queue Sal Index

In [ ]:
## Validate input data

# Validates that ndays is a string that can either a number of hours (e.g. '6h') or a number of days (e.g. '3d')
if not ndays[-1] in ["h", "d"]:
    raise ValueError("ndays must end in either 'h' or 'd'")

# Assert cliente name is either summit_efd or usdf_efd
assert client_name in [
    "summit_efd",
    "usdf_efd",
], "client_name must be either 'summit_efd' or 'usdf_efd'"

# Assert path is a string
assert isinstance(path, str), "path must be a string"

# Assert salIndex is an integer either 1, 2 or 3
assert salIndex in [1, 2, 3], "salIndex must be an integer either 1, 2 or 3"

# Script Log Inspection Notebook

## Overview

This notebook is designed to query and analyze script logs for specific executions.
It allows users to inspect detailed log messages for a given script path and Script Queue (Sal Index), providing insights into script behavior, potential issues, and tracebacks.

## Inputs

The inputs for this notebook are:

1. **`ndays`:**
   Allows selecting the number of days (**or hours**) to query the script queue logs (e.g., '1h', '10h', '0.5d', '1d', '10d', etc.).

2. **`EFD Client`:**
   Allows switching between available EFD clients (such as `usdf_efd` or `summit_efd`) if you need to query data from different environments.
   If analyzing real-time data, the `summit_efd` might be the preferred client.

3. **`Script Path`:**
   Lets you select the script path for which you want to query the logs (e.g., `maintel/m1m3/check_actuators.py`, `maintel/track_target.py`, `auxtel/track_target.py`, etc.).

4. **`SalIndex`:**
   Lets you select the Script Queue `SalIndex` to query logs from different queues:
   - `1`: Simonyi Telescope Queue
   - `2`: AuxTel Queue
   - `3`: OCS Queue


In [ ]:
# Imports and basic setup

import asyncio
import re
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import (
    ColumnDataSource,
    CustomJS,
    DataTable,
    Div,
    Select,
    TableColumn,
)
from IPython.display import HTML, display
from lsst_efd_client import EfdClient

In [ ]:
## Utility Functions

### Query and Filter Script Queue Logs


# This function converts user-defined time ranges into hours for querying the EFD.
def convert_to_hours(past_time):
    """
    Convert a string that can be either a number of hours (e.g. '6h')
    or a number of days (e.g. '3d') to total number of hours (int).
    """
    match = re.match(r"(\d+)([dh]?)", past_time)
    if match:
        value, unit = match.groups()
        value = int(value)
        if unit == "d":
            return value * 24
        else:
            return value
    else:
        raise ValueError(
            "Invalid time format. Please use a format "
            "like '6h' for hours or '3d' for days."
        )


# A function to retrieve script queue logs for a given time range and client.
async def query_script_queue_logs(
    start_time_str: str, end_time_str: str, client_name="usdf_efd"
):
    """
    Queries the log messages related to the script queue from the EFD.

    Parameters
    ----------
    start_time_str : str
        Start time in ISO format, e.g. "2024-11-04T12:00:00".
    end_time_str : str
        End time in ISO format, e.g. "2024-11-04T13:00:00".
    client_name : str, optional
        Name of the EFD client. Defaults to "usdf_efd".

    Returns
    -------
    pd.DataFrame
        DataFrame with script queue logs in the given time window.
    """
    # Convert string times to astropy Time
    start = Time(start_time_str, format="isot", scale="utc")
    end = Time(end_time_str, format="isot", scale="utc")

    # Create the EFD client
    possible_clients = ["summit_efd", "usdf_efd"]
    if client_name not in possible_clients:
        print(f"Invalid client name. Possible clients: {possible_clients}")
        return None

    client = EfdClient(client_name)
    script_logs = await client.select_time_series(
        topic_name="lsst.sal.ScriptQueue.logevent_script",
        fields="*",
        start=start,
        end=end,
    )

    return script_logs


# This function filters raw logs to focus on a specific script and processes the data into a clean format.
def filter_and_process_queue_logs(
    script_logs, path="maintel/m1m3/check_actuators.py", salIndex=1
):
    """
    Filters and processes the script logs DataFrame for a specific  path and salIndex.

    Parameters
    ----------
    script_logs : pd.DataFrame
        DataFrame containing the script logs.

    Returns
    -------
    pd.DataFrame
        Processed DataFrame containing only relevant logs for the check_actuators script.
    """
    processState_mapping = {
        0: "UNKNOWN",
        1: "LOADING",
        2: "CONFIGURED",
        3: "RUNNING",
        4: "DONE",
        5: "LOADFAILED",
        6: "CONFIGURE_FAILED",
        7: "TERMINATED",
    }

    scriptState_mapping = {
        0: "UNKNOWN",
        1: "UNCONFIGURED",
        2: "CONFIGURED",
        3: "RUNNING",
        4: "PAUSED",
        5: "ENDING",
        6: "STOPPING",
        7: "FAILING",
        8: "DONE",
        9: "STOPPED",
        10: "FAILED",
        11: "CONFIGURE_FAILED",
    }

    df_script_logs = pd.DataFrame(script_logs)

    # Filter logs for the specific script path and salIndex
    df_filtered_logs = df_script_logs[
        (df_script_logs["path"] == path) & (df_script_logs["salIndex"] == salIndex)
    ]

    # Reindex by private_rcvStamp, convert to UTC
    df_filtered_logs = df_filtered_logs.set_index("private_rcvStamp")
    df_filtered_logs.index = pd.to_datetime(df_filtered_logs.index, unit="s")
    df_filtered_logs.index = df_filtered_logs.index - timedelta(seconds=37)

    # Convert all columns starting with 'timestamp' to datetime, also shift TAI->UTC
    timestamp_columns = [
        col for col in df_filtered_logs.columns if col.startswith("timestamp")
    ]
    for col in timestamp_columns:
        df_filtered_logs[col] = pd.to_datetime(df_filtered_logs[col], unit="s")
        df_filtered_logs[col] = df_filtered_logs[col] - timedelta(seconds=37)

    # Map numeric process/script states to strings
    df_filtered_logs["processState_str"] = df_filtered_logs["processState"].map(
        processState_mapping
    )
    df_filtered_logs["scriptState_str"] = df_filtered_logs["scriptState"].map(
        scriptState_mapping
    )

    # Remove unneeded columns
    columns_to_remove = [
        "blockId",
        "blockSize",
        "cmdId",
        "executionId",
        "private_efdStamp",
        "private_kafkaStamp",
        "private_origin",
        "private_revCode",
        "private_sndStamp",
        "private_seqNum",
        "scriptBlockIndex",
        "private_identity",
        "processState",
        "scriptState",
    ]
    df_filtered_logs.drop(columns=columns_to_remove, inplace=True, errors="ignore")

    # Sort from most recent to oldest
    df_filtered_logs.sort_index(ascending=False, inplace=True)

    return df_filtered_logs


# Extracts details such as start time, end time, and final status for each execution.
def extract_execution_details(df_check_actuators_log):
    """
    Extracts execution details from the script logs DataFrame.

    Parameters
    ----------
    df_check_actuators_log : pd.DataFrame
        DataFrame containing the filtered script logs.

    Returns
    -------
    pd.DataFrame
        DataFrame containing execution times, durations, and final statuses.
    """
    # Initialize an empty list to store execution details
    execution_data = []

    # Loop over each unique scriptSalIndex
    for script_sal_index in df_check_actuators_log["scriptSalIndex"].unique():
        df_sal = df_check_actuators_log[
            df_check_actuators_log["scriptSalIndex"] == script_sal_index
        ]
        df_sal = df_sal.sort_index()  # Ensure chronological order

        # Calculate start and end times for each execution
        start_time = df_sal["timestampProcessStart"].min()
        end_time = df_sal["timestampProcessEnd"].max()

        # Get the final process and script status
        final_process_status = df_sal["processState_str"].iloc[-1]
        final_script_status = df_sal["scriptState_str"].iloc[-1]

        execution_data.append(
            {
                "path": df_sal["path"].iloc[0],
                "scriptSalIndex": script_sal_index,
                "start_time": start_time,
                "end_time": end_time,
                "FinalProcessStatus": final_process_status,
                "FinalScriptStatus": final_script_status,
            }
        )

    # Create the DataFrame
    df_executions = pd.DataFrame(execution_data)

    if not df_executions.empty:
        # Calculate duration in minutes
        df_executions["duration_minutes"] = (
            df_executions["end_time"] - df_executions["start_time"]
        ).dt.total_seconds() / 60.0

        # Format durations to .2f
        df_executions["duration_minutes"] = df_executions["duration_minutes"].apply(
            lambda x: "{:.2f}".format(x)
        )

        # Reorder columns
        cols = [
            "path",
            "scriptSalIndex",
            "start_time",
            "end_time",
            "duration_minutes",
            "FinalProcessStatus",
            "FinalScriptStatus",
        ]
        df_executions = df_executions[cols]

        # Sort by start time in descending order
        df_executions = df_executions.sort_values("start_time", ascending=False)
    else:
        print("No executions found.")

    return df_executions

In [ ]:
## Parameters Review

#### This cell prints the user-defined parameters to ensure correctness before querying.

print("Parameters set:")
print(f"  Number of days/hours: {ndays}")
print(f"  EFD Client: {client_name}")
print(f"  Sal Index: {salIndex}")
print(f"  Script Path: {path}")

## Query Script Executions

This section queries the EFD for script executions, processes the results and displays details for the executions found.

In [ ]:
past_hours = convert_to_hours(ndays)

now_utc = datetime.utcnow()
end_dt = now_utc
start_dt = now_utc - timedelta(hours=past_hours)

start_str = start_dt.isoformat()
end_str = end_dt.isoformat()

print(f"Querying script executions for {path} from {start_str} to {end_str}...")
script_logs = await query_script_queue_logs(start_str, end_str, client_name)

if script_logs.empty:
    print("No script executions found.")
else:
    df_logs = filter_and_process_queue_logs(script_logs, path=path, salIndex=salIndex)
    df_executions = extract_execution_details(df_logs)

# Display executions summary
if df_executions.empty:
    raise ValueError(
        "No executions found in the last {:.1f} days.".format(past_hours / 24.0)
    )
else:
    print("Executions found in the last {:.1f} days:".format(past_hours / 24.0))
    display(HTML(df_executions.to_html(escape=False)))

### Interactive Log Inspection App

**Functionality:**

- **Dropdown:**
   Select a specific Script Queue `SalIndex` to inspect logs for executions.

- **Log Messages Table:**
   Displays a tabular view of the log messages for the selected execution, including columns for `filePath`, `functionName`, `message`, and truncated `traceback`.

- **Traceback Viewer:**
   Clicking on a row in the table shows the full traceback in a separate viewer if available.

**Output:**

Upon selecting a specific `SalIndex` and interacting with the log inspection table:

1. **`Log Messages Table`:**
   - A tabular format showing the log fields, such as `filePath`, `functionName`, `message`, and `traceback`.
   - Allows row selection to view full tracebacks.

2. **`Traceback Viewer`:**
   - Displays the full traceback for the selected log message.
   - If no traceback is present, the viewer indicates this to the user.

This interactive app makes it easy to explore and analyze script logs, facilitating efficient debugging and understanding of script behaviors across different queues.

In [ ]:
# A function to truncate a traceback string if it's too long.
def truncate_traceback(tb_str, length=120):
    """
    Return a truncated version of tb_str if it's longer than 'length'.
    Append '... [click row for full]' at the end to signal more content.
    """
    if not tb_str:
        return ""
    tb_str = str(tb_str)  # Ensure it's a string
    if len(tb_str) <= length:
        return tb_str
    else:
        return tb_str[:length] + "... [click row for full]"


# A function to build log data for each execution by querying the SAL log messages.
async def build_log_data_by_salindex(df_executions, client_name="usdf_efd"):
    """
    For each row in df_executions, query lsst.sal.Script.logevent_logMessage
    in the time range [start_time, end_time], and build a dictionary
    mapping salIndex -> {filePath, functionName, message, traceback}.

    Parameters
    ----------
    df_executions : pd.DataFrame
        Must include columns "scriptSalIndex", "start_time", and "end_time".
    client_name : str
        EFD client name ("usdf_efd", "summit_efd", etc.).

    Returns
    -------
    dict
        A dictionary keyed by SAL index (str), with values as column-data dict:
        {
            "filePath":     [...],
            "functionName": [...],
            "message":      [...],
            "traceback":    [...]
        }
    """
    # Create an EFD client
    client = EfdClient(client_name)

    # Prepare a dict to store all log messages, keyed by SAL index
    log_data = {}

    for _, row in df_executions.iterrows():
        sal_idx = row["scriptSalIndex"]
        start_time = row["start_time"]
        end_time = row["end_time"]

        # Convert to ISO strings if needed
        start_str = start_time.isoformat()
        end_str = end_time.isoformat()

        # Query the 'Script.logevent_logMessage' topic for [start_time, end_time]
        log_msgs = await client.select_time_series(
            "lsst.sal.Script.logevent_logMessage",
            fields="*",
            start=Time(start_str, format="isot", scale="utc"),
            end=Time(end_str, format="isot", scale="utc"),
        )

        # If empty, store an empty dictionary structure
        if len(log_msgs) == 0:
            log_data[str(sal_idx)] = {
                "filePath": [],
                "functionName": [],
                "message": [],
                "traceback": [],
            }
            continue

        # Convert results to DataFrame, remove extraneous columns
        df_msgs = pd.DataFrame(log_msgs)
        drop_cols = [
            "private_efdStamp",
            "private_identity",
            "private_kafkaStamp",
            "private_origin",
            "private_rcvStamp",
            "private_revCode",
            "private_seqNum",
            "private_sndStamp",
            "process",
            "salIndex",
            "timestamp",
        ]
        df_msgs.drop(columns=drop_cols, inplace=True, errors="ignore")

        # Ensure the required columns exist (some logs might be missing them)
        for needed in ["filePath", "functionName", "message", "traceback"]:
            if needed not in df_msgs.columns:
                df_msgs[needed] = ""

        # Create a truncated traceback column
        df_msgs["tracebackFull"] = df_msgs["traceback"]
        df_msgs["traceback"] = df_msgs["tracebackFull"].apply(truncate_traceback)

        # Restrict to final columns
        df_msgs = df_msgs[
            ["filePath", "functionName", "message", "traceback", "tracebackFull"]
        ]

        # Convert to dict-of-lists
        log_data[str(sal_idx)] = df_msgs.to_dict(orient="list")

    return log_data

In [ ]:
###############################################################################
# Complete Bokeh App Cell
###############################################################################


# 1) Activate Bokeh in the notebook
output_notebook()

# ------------------------------------------------------------------------------
# 2) Build the dictionary log_data_by_salindex using your existing df_executions
#    and the two helper functions you've provided:
#    (a) truncate_traceback(tb_str, length=120)
#    (b) build_log_data_by_salindex(df_executions, client_name="usdf_efd")
# ------------------------------------------------------------------------------

# Check that df_executions is not empty
if df_executions.empty:
    raise ValueError(
        "No executions found. You may need to change notebook input parameters."
    )

# Build the log data dictionary
log_data_by_salindex = await build_log_data_by_salindex(df_executions, client_name)

# 3) Pick the first SAL index in df_executions as our initial selection.
initial_salindex = df_executions["scriptSalIndex"].iloc[0]

# Grab the corresponding data dict (or an empty fallback).
initial_data = log_data_by_salindex.get(
    str(initial_salindex),
    {
        "filePath": [],
        "functionName": [],
        "message": [],
        "traceback": [],
        "tracebackFull": [],
    },
)

# Create the ColumnDataSource with that initial data
source = ColumnDataSource(initial_data)

# 4) Define the columns for the main DataTable.
#    We show the truncated traceback in "traceback" (already truncated in the dict).
columns = [
    TableColumn(field="filePath", title="filePath", width=600),
    TableColumn(field="functionName", title="functionName", width=200),
    TableColumn(field="message", title="message", width=700),
    TableColumn(field="traceback", title="traceback", width=200),
]

data_table = DataTable(
    source=source,
    columns=columns,
    width=1700,
    height=1200,
    # This determines how row selection is made:
    #   "checkbox" -> user must tick a box at left to select the row
    #   "row" -> user clicks on the row
    selectable="checkbox",
    # Hide the default numerical index column on the left
    index_position=None,
)

# 5) Create a Div to display the FULL traceback text
traceback_div = Div(
    text="Select a row to see its full traceback here, if the traceback is not empty...",
    width=1700,
    height=600,
    styles={"overflow": "auto", "border": "2px solid gray", "padding": "5px"},
)

# 6) Create a dropdown to pick a different SAL index
salindex_select = Select(
    title="Inspect Logs (SalIndex):",
    value=str(initial_salindex),
    options=[str(idx) for idx in df_executions["scriptSalIndex"]],
    width=130,
)

# When the user changes the dropdown, update the entire table + clear the Div
salindex_select.js_on_change(
    "value",
    CustomJS(
        args=dict(source=source, data_dict=log_data_by_salindex, tb_div=traceback_div),
        code="""
            const selectedSalIndex = cb_obj.value;
            const newData = data_dict[selectedSalIndex] || {
                filePath:      [],
                functionName:  [],
                message:       [],
                traceback:     [],
                tracebackFull: []
            };
            source.data = newData;
            source.change.emit();
            tb_div.text = "Select a row to see its full traceback here, if the traceback is not empty...";
        """,
    ),
)

# 7) When a row is selected, show the FULL traceback in the Div
source.selected.js_on_change(
    "indices",
    CustomJS(
        args=dict(source=source, tb_div=traceback_div),
        code="""
            const inds = source.selected.indices;
            if (inds.length === 0) {
                tb_div.text = "No row selected. Select a row with an existing traceback if you want to expand its content.";
                return;
            }
            // We take the first selected row for demonstration
            const i = inds[0];
            const tbFull = source.data.tracebackFull[i];

            if (!tbFull || tbFull.trim().length === 0) {
                tb_div.text = "No traceback for this row.";
            } else {
                tb_div.text = `<pre>${tbFull}</pre>`;
            }
        """,
    ),
)

# 8) Layout: a column with the dropdown, then table, then the traceback div
layout = column(
    salindex_select,
    row(data_table),
    row(traceback_div),
)

# 9) Show the final app in the notebook
show(layout)